<a href="https://colab.research.google.com/github/nitish-raj/Kaggle-Competitions/blob/main/spp-boosting-fe-optuna-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><p style="text-align: center;"> <u> Song Popularity Prediction </u>  </p> </h1>
<h3><p style="text-align: center;"> <i> Tabular The Data Is </i>  </p> </h3>

<img src="https://i.gifer.com/9mhx.gif" width="500" />

#### *Yeah, you guessed it right, my favourite musician*

In [4]:
!pip install kaggle
!pip install optuna
!pip install category_encoders

     |████████████████████████████████| 82 kB 440 kB/s 


In [5]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,roc_auc_score
from xgboost import XGBClassifier,XGBRegressor,plot_importance,XGBRFRegressor

from scipy.stats import mode,boxcox,skew

import matplotlib.pyplot as plt
import optuna
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,OneHotEncoder,PowerTransformer
from sklearn.preprocessing import RobustScaler,MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer,KNNImputer,IterativeImputer
from sklearn.feature_selection import SelectFromModel

from category_encoders import target_encoder
import seaborn as sns
import gc
import sys,os

from scipy.spatial import distance
gc.enable()

pd.set_option('display.float_format', lambda x: '%.3f' % x)

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [8]:
!kaggle competitions download -c song-popularity-prediction

100% 849k/849k [00:00<00:00, 6.23MB/s]

  0% 0.00/67.3k [00:00<?, ?B/s]
100% 67.3k/67.3k [00:00<00:00, 61.0MB/s]
 30% 1.00M/3.33M [00:00<00:00, 5.06MB/s]
100% 3.33M/3.33M [00:00<00:00, 12.9MB/s]


In [11]:
!unzip '/content/train.csv.zip'
!unzip '/content/test.csv.zip'
!unzip '/content/sample_submission.csv.zip'

Archive:  /content/train.csv.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                
unzip:  cannot find or open /content/sample_submission.csv.zip, /content/sample_submission.csv.zip.zip or /content/sample_submission.csv.zip.ZIP.


In [16]:
!rm '/content/test.csv.zip'
!rm '/content/train.csv.zip'

rm: cannot remove '/content/train.csv.zip': No such file or directory


### 1. Read Train, Test and Submission files

In [17]:
# Read Train, Test and Sample Submission Files
def read_data():
    df_train = pd.read_csv("/content/train.csv")
    df_test = pd.read_csv("/content/test.csv")
    df_submission = pd.read_csv("/content/sample_submission.csv")
    return df_train,df_test,df_submission

In [18]:
# Read datasets
df_train,df_test,df_submission = read_data()

#### Let's see how you look.. any missing values.. ANY?

In [19]:
df_train.isna().sum()

id                     0
song_duration_ms    4101
acousticness        3992
danceability        4026
energy              3975
instrumentalness    3985
key                 4065
liveness            4086
loudness            3957
audio_mode             0
speechiness            0
tempo                  0
time_signature         0
audio_valence          0
song_popularity        0
dtype: int64

In [20]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                40000 non-null  int64  
 1   song_duration_ms  35899 non-null  float64
 2   acousticness      36008 non-null  float64
 3   danceability      35974 non-null  float64
 4   energy            36025 non-null  float64
 5   instrumentalness  36015 non-null  float64
 6   key               35935 non-null  float64
 7   liveness          35914 non-null  float64
 8   loudness          36043 non-null  float64
 9   audio_mode        40000 non-null  int64  
 10  speechiness       40000 non-null  float64
 11  tempo             40000 non-null  float64
 12  time_signature    40000 non-null  int64  
 13  audio_valence     40000 non-null  float64
 14  song_popularity   40000 non-null  int64  
dtypes: float64(11), int64(4)
memory usage: 4.6 MB


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                10000 non-null  int64  
 1   song_duration_ms  9043 non-null   float64
 2   acousticness      9050 non-null   float64
 3   danceability      9024 non-null   float64
 4   energy            8945 non-null   float64
 5   instrumentalness  9000 non-null   float64
 6   key               9008 non-null   float64
 7   liveness          8994 non-null   float64
 8   loudness          8974 non-null   float64
 9   audio_mode        10000 non-null  int64  
 10  speechiness       10000 non-null  float64
 11  tempo             10000 non-null  float64
 12  time_signature    10000 non-null  int64  
 13  audio_valence     10000 non-null  float64
dtypes: float64(11), int64(3)
memory usage: 1.1 MB


**Insight**: Both Test and Train have same columns which have NULL values .. Nice

In [ ]:
df_train.describe().T

In [ ]:
df_test.describe().T

### What does each variable mean?
> - **Instrumentalness:** This value represents the amount of vocals in the song. The closer it is to 1.0, the more instrumental the song is.
> - **Acousticness:** This value describes how acoustic a song is. A score of 1.0 means the song is most likely to be an acoustic one.
> - **Liveness:** This value describes the probability that the song was recorded with a live audience. According to the official documentation “a value above 0.8 provides strong likelihood that the track is live”.
> - **Speechiness:** “Speechiness detects the presence of spoken words in a track”. If the speechiness of a song is above 0.66, it is probably made of spoken words, a score between 0.33 and 0.66 is a song that may contain both music and words, and a score below 0.33 means the song does not have any speech.
> - **Energy:** “(energy) represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy”.
> - **Danceability:** “Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable”.
> - **Valence:** “A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry)”
> - **Song Duration ms :** Duration of song in milliseconds
> - **Audio Mode :** No specific description
> - **Tempo :** Tempo (Italian for "time"; plural tempos, or tempi from the Italian plural) is the speed or pace of a given piece.For example, a tempo of 60 beats per minute signifies one beat per second, while a tempo of 120 beats per minute is twice as rapid, signifying one beat every 0.5 seconds
> - **Time Signature :** The time signature indicates how many counts are in each measure and which type of note will receive one count. The top number is commonly 2, 3, 4, or 6.The bottom number is either 4 or 8. Simple time signatures divide music into groups of 2 and compound divide music into groups of 3.
> - **Loudness:** Loudness measures the decibel level of a song. Decibels are relative to a reference value, so songs with lower loudness values are quieter relative to the reference value of 0.
> - **Danceability:** Danceability quantifies how suitable a track is for dancing based on a combination of musical elements, like tempo, rhythm, and beat. Songs with higher danceability have stronger and more regular beats.Like acousticness, danceability is measured on a scale of 0 (low danceability) to 100 (high danceability).

In [ ]:
cont_cols = [col for col in df_train.columns if col not in ['id','song_popularity','audio_mode','time_signature','key']]
cat_cols = ['audio_mode','time_signature','key']

In [ ]:
print(f'% Distribution of Song Popularity:\n{(df_train.song_popularity.value_counts())/len(df_train.song_popularity)*100}')

In [ ]:
df_train.key.value_counts()

In [ ]:
df_train.audio_mode.value_counts()

In [ ]:
df_train.time_signature.value_counts()

### 2. Plot the distribution of data for each column

![Alt text](https://media.makeameme.org/created/if-you-didnt-58c83d63de.jpg)

In [ ]:
for col in cat_cols:
    df_train.hist(col,grid=False,bins = 12)

> **I NEED SOME CHARTS.. NOW**

Hmm... Below plots will show the distribution of data against our target variables.. Please be nice

In [ ]:
for col in cont_cols:
    sns.displot(data=df_train, x=col,hue="song_popularity",kind = "kde", fill=True)

> Let's check box plots for those.. outliers

In [ ]:
for col in cont_cols:
    plt.figure()
    sns.boxplot(data = df_train,x='song_popularity',y = col)

### I know my visualization skills need a serious upgrade

#### *Working..... On it........*

In [ ]:
# Check correlation between the columns
corr = df_train.drop(['id','song_popularity'],axis=1).corr()
cm = sns.light_palette("green", as_cmap=True)
cm = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
corr.style.background_gradient(cmap=cm)

> **Let's Create a function below to Scale our data**

In [ ]:
# Function to Scale and transform dataset
def data_scaler_fit(option,df):
    if option == 1:
        transformer = StandardScaler().fit(df)
    if option == 2 :
        transformer = RobustScaler().fit(df)
    if option == 3 :
        transformer = MinMaxScaler().fit(df)
    if option == 4 :
        transformer = PowerTransformer(method = 'yeo-johnson').fit(df)
    return transformer

> **A Function to remove outlier, the standard way**

In [ ]:
# Function to Remove outliers
def remove_outliers(x,method):
    if method == 'mean':
        upper_limit = x.mean() + (3*x.std())
        lower_limit = x.mean() - (3*x.std())
        return np.where(x > upper_limit,upper_limit,np.where(x <lower_limit,lower_limit,x))
    elif method == 'median':
        upper_limit = x.median() + (1.5*x.quantile(0.75))
        lower_limit = x.median() - (1.5*x.quantile(0.25))
        return np.where(x > upper_limit,upper_limit,np.where(x <lower_limit,lower_limit,x))
    else:
        return x

In [ ]:
def imputations(impute_method,method=None):
    if impute_method == 'knn':
        imputer = KNNImputer(n_neighbors = 10,weights = 'distance')
    if impute_method == 'iter':
        imputer = IterativeImputer(max_iter=20)
    if impute_method == 'simple':
        imputer = SimpleImputer(strategy=method)
    if impute_method == 'lgbm':
        if not os.path.exists("kuma_utils/"):
            !git clone https://github.com/analokmaus/kuma_utils.git
        sys.path.append("kuma_utils/")
        from kuma_utils.preprocessing.imputer import LGBMImputer
        imputer = LGBMImputer(n_iter=300, verbose=False)
    return imputer

### 3. Let's Transform those features, one at a time

> 1. Column **Key**,**audio_mode**,**time_signature**  seems to be categorical, hence replacing null values using KNN
> 2. Convert **time_signature** from milliseconds to minutes
> 3. Fill Rest of the columns which has null values using mean of the columns (Might replace with something better)
> 4. Use Boxcox transform to reduce skewness in continuous columns
> 5. Finally using StandarScaler on all continuous columns : *It transforms the data in such a manner that it has mean as 0 and standard deviation as 1*

In [ ]:
def feature_transform(df,option,method):
    ids = df.id.values.tolist()
    
    # Replace missing values for continuous columns
    impute = imputations(impute_method)
    
    df_temp = pd.DataFrame(impute.fit_transform(pd.concat([df[cat_cols],df[cont_cols]],axis = 1)))
    
    df_temp.columns = cat_cols + cont_cols
    
    df_cat = df_temp[cat_cols].copy()
    df_cont = df_temp[cont_cols].copy()
    
    # Decreasing data skewness for continuos variables using BoxCox
    df_cont['golden_ratio'] = (df_cont['song_duration_ms'] / 1000) * 0.618033
    df_cont['song_duration_ms'] = boxcox(df_cont.song_duration_ms/60000)[0]
    
    df_cont['instrumentalness_main'] = np.where(df_cont['instrumentalness']<=0.01,df_cont['instrumentalness'],0)
    df_cont['instrumentalness_side'] = np.where(df_cont['instrumentalness']>0.2,df_cont['instrumentalness'],0)
    df_cont.drop('instrumentalness',axis =1 ,inplace=True)
    df_cont['instrumentalness_side'] = boxcox((df_cont.instrumentalness_side)+0.005)[0]
    #df_cont['instrumentalness_main'] = boxcox((df_cont.instrumentalness_main)+0.2)[0]
    
    
    df_cont['acousticness'] = boxcox(df_cont.acousticness + 0.2)[0]
    
    df_cont['liveness_main'] = np.where(df_cont['liveness']<0.8,df_cont['liveness'],0)
    df_cont['liveness_side'] = np.where(df_cont['liveness']>=0.8,df_cont['liveness'],0)
    df_cont.drop('liveness',axis =1 ,inplace=True)
    #df_cont['liveness']  = boxcox(df_cont.liveness + 0.04)[0]    
    
    df_cont['loudness'] = boxcox(df_cont.loudness + 100)[0] 
    df_cont['energy'] = boxcox(df_cont.energy+0.2)[0]
    df_cont['danceability'] = boxcox(df_cont.danceability)[0]
  
    df_cont['speechiness'] = boxcox(df_cont.speechiness)[0]
    df_cont['tempo'] = boxcox(df_cont.tempo)[0]
    
    # Normalise Continuous columns
    transformer = data_scaler_fit(option,df_cont)
    df_cont = pd.DataFrame(transformer.transform(df_cont.apply(lambda x: remove_outliers(x,method))))
    
    df_cont.columns = [x for x in cont_cols if (x not in ['instrumentalness','liveness'])] + ['golden_ratio','instrumentalness_main','instrumentalness_side','liveness_main','liveness_side']
    
    df = pd.concat([df_cont,df_cat.reset_index(drop=True)],axis = 1)
    
    df['id'] = ids
    
    return df

In [ ]:
# Initialize Static values
random_state = 151
early_stopping_rounds=1500
verbose = 2000
n_estimators = 15000
n_splits = 5
method = 'mean'
scaling_option = 2
impute_method = 'lgbm'

### 4. Using Optuna for Hyperparameter Tuning

In [ ]:
df_train,df_test,df_submission = read_data()

y = df_train.song_popularity.copy()
X = df_train.drop(['song_popularity'],axis = 1)


def objective(trial):
    # XGBoost parameters
    
    gc.collect()
    
    final_valid_predictions = {}
    
    scores = []
    auc_score = []
    
    params = {
        "objective": "reg:squaredlogerror",
        "n_estimators": n_estimators,
        "max_depth": trial.suggest_int("max_depth", 2, 15),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.001, 0.5),
        "colsample_bytree": trial.suggest_loguniform("colsample_bytree", 0.2, 0.9),
        "subsample": trial.suggest_loguniform("subsample", 0.2, 0.9),
        "alpha": trial.suggest_loguniform("alpha", 0.01, 100.0),
        "lambda": trial.suggest_loguniform("lambda", 0.01, 100.0),
        "gamma": trial.suggest_loguniform("lambda", 0.01, 100.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 10, 100),
        "n_jobs": -1,
        "tree_method": "gpu_hist",
        "predictor" : "gpu_predictor",
    }
    
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for fold, (idx_train, idx_valid) in enumerate(cv.split(X,y)):
        xtrain, ytrain = X.iloc[idx_train], y[idx_train]
        xvalid, yvalid = X.iloc[idx_valid], y[idx_valid]

        # Store IDs of validation Dataset
        valid_ids = xvalid.id.values.tolist()

        #Save a copy of yvalid
        true_valid = yvalid

        n_class = len(np.unique(ytrain))

        xtrain = feature_transform(xtrain,scaling_option,method)
        xvalid = feature_transform(xvalid,scaling_option,method)
        
        xtrain = xtrain.drop('id',axis = 1)
        xvalid = xvalid.drop('id',axis = 1)
        
        
        model = XGBRegressor(
            random_state = random_state,
            sampling_method = 'gradient_based',
            use_label_encoder=False,
            eval_metric = ['aucpr','auc'],
            **params
        )
        model.fit(xtrain, ytrain,early_stopping_rounds=early_stopping_rounds, eval_set=[(xvalid, yvalid)], verbose=verbose)

        preds_valid = model.predict(xvalid)
        
        final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))

        auc_scr = roc_auc_score(true_valid, preds_valid)

        auc_score.append(auc_scr)

        print(f"Fold {fold+1} || AUC : {auc_scr} || Mean AUC : {np.mean(auc_score)}")
    
    return roc_auc_score(y.to_numpy(), np.array(sorted(final_valid_predictions.items()))[:,1])

![alt text](https://miro.medium.com/max/900/1*80wf6AeqTLD9ntyFxYMuLw.jpeg)

### Keep Calm .. Optuna is Studying....

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

#### Print best parameters ..

In [ ]:
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

### 5. Training XGB Model based on best Parameters

In [ ]:
# Read DATA Again
df_train,df_test,df_submission = read_data()

# Seperate X and y
y = df_train.song_popularity.copy()
X = df_train.drop(['song_popularity'],axis = 1)


# Transform Test Dataset
df_test = feature_transform(df_test,scaling_option,method)


final_test_predictions = []
final_valid_predictions = {}

scores = []
auc_score = []

cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

for fold, (idx_train, idx_valid) in enumerate(cv.split(X,y)):
    xtrain, ytrain = X.iloc[idx_train], y[idx_train]
    xvalid, yvalid = X.iloc[idx_valid], y[idx_valid]
    
    xtest = df_test.copy().drop('id',axis = 1)

    # Store IDs of validation Dataset
    valid_ids = xvalid.id.values.tolist()
    
    #Save a copy of yvalid
    true_valid = yvalid
    
    n_class = len(np.unique(ytrain))
    
    xtrain = feature_transform(xtrain,scaling_option,method)
    xvalid = feature_transform(xvalid,scaling_option,method)
    
    xtrain = xtrain.drop('id',axis = 1)
    xvalid = xvalid.drop('id',axis = 1)
    
    
    static = {
        "n_estimators": n_estimators,
        "objective": "reg:squaredlogerror",
        "random_state": random_state,
        "n_jobs": -1,
        #"tree_method": "gpu_hist",
        #"predictor" : "gpu_predictor",
        "eval_metric" : ['aucpr','auc'],
        #"sampling_method" : 'gradient_based',
        "use_label_encoder" : False,
    }
    
    params = dict(static)

    params.update(study.best_params)
    

    model = XGBRegressor(
        **params
    )
    model.fit(xtrain, ytrain,early_stopping_rounds=early_stopping_rounds, eval_set=[(xtrain, ytrain),(xvalid, yvalid)], verbose=verbose)

    preds_valid = model.predict(xvalid)

    test_preds = model.predict(xtest)

    final_test_predictions.append(test_preds)

    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))

    auc_scr = roc_auc_score(true_valid, preds_valid)
    
    auc_score.append(auc_scr)
    
    print('_'*65)
    
    print(f"Fold {fold+1}  || AUC : {auc_scr} || Mean AUC : {np.mean(auc_score)}")
    
    print('_'*65)
    
    print('\n')
    
    gc.collect()

In [ ]:
#Plot Feature Importance
plt.figure(figsize=(20, 15),dpi=80)
plot_importance(model)
plt.show()

In [ ]:
# retrieve performance metrics
results = model.evals_result()
epochs = len(results['validation_0']['aucpr'])
x_axis = range(0, epochs)
# plot AUCPR
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['aucpr'], label='Train')
ax.plot(x_axis, results['validation_1']['aucpr'], label='Valid')
ax.legend()
plt.ylabel('AUCPR Score')
plt.title('XGBoost AUCPR')
plt.show()
# plot AUC
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['auc'], label='Train')
ax.plot(x_axis, results['validation_1']['auc'], label='Valid')
ax.legend()
plt.ylabel('AUC Score')
plt.title('XGBoost AUC')
plt.show()

### 6. Submit Score

In [ ]:
df_submission.song_popularity = np.mean(np.column_stack(final_test_predictions), axis=1)
df_submission.columns = ["id", "song_popularity"]
df_submission.to_csv("submission.csv", index=False)

***Do upvote if you found this useful*** 

PS: I am still working on improving this notebook and Model, so stay tuned!!!

<p style="text-align: center" >
<img src= "https://media.giphy.com/media/DMHEccCwpNxCQBZlvQ/giphy.gif"
</p>